In [1]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from joblib import dump,load
from sklearn.feature_selection import RFECV

In [2]:
#读数据标准化划分数据集
df = pd.read_csv('../my_data.csv')
X = df.iloc[:, 1:58]
Y = df.iloc[:, 58:]
X_to_select = ['Sn_frac', 'A_Ionic_radius', 'A/X_Ionic_radius',
 'B/X_Ionic_radius', 'B_Ionization energy(kJ/mol)', 'X_Ionization energy(kJ/mol)',
 'B_Electron_affinity(kJ/mol)', 'B_Valence_electrons', 'B_Heat_of_vap(kJ/mol)', 
 'B_Electronegativity']
X=X[X_to_select]
Y=Y['SLME @ 5um']
names=df.columns
names=['SHAP value of '+x for x in names]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=10)
print("X_train's shape is", X_train.shape,"; y_train's shape is", y_train.shape)
print("X_test's shape is", X_test.shape,"; y_test's shape is",y_test.shape)
scaler = StandardScaler()
scaler.fit(X_train)
X_train_stand = scaler.transform(X_train)
X_test_stand = scaler.transform(X_test)

X_train's shape is (390, 10) ; y_train's shape is (390,)
X_test's shape is (98, 10) ; y_test's shape is (98,)


In [3]:
##gridsearch and cv select model##
param_grid={ 'learning_rate': [0.01,0.05,0.1,0.15],
              'n_estimators': [15,50,100,200],
              'max_depth': [4,5,6,7,8],
              'min_child_weight': [3,4,6,7],
              'subsample': [0.7,0.8,0.9,1],
              'colsample_bytree': [0.3,0.5,0.6,0.7],
              'gamma': [0,0.1,0.2,0.3,0.4],
                'reg_alpha': [0,0.1,0.2,0.3],
                'reg_lambda': [1,2,5]
            }

In [4]:
xgb = XGBRegressor()
model = GridSearchCV(estimator =xgb,scoring='neg_root_mean_squared_error', 
                     param_grid = param_grid,cv=3, verbose=3,n_jobs =-1,return_train_score=True)
model.fit(X_train, y_train)
print(model.cv_results_)
dfresult=pd.DataFrame(model.cv_results_)
dfresult.to_csv('SLME_13y32.csv')
print(dfresult)
print(model.best_params_)
print(model.scorer_)
print(model.best_score_)
bestmodel=model.best_estimator_

Fitting 3 folds for each of 307200 candidates, totalling 921600 fits


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



        mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0            0.011367      0.001723         0.002004    2.809790e-06   
1            0.008776      0.000882         0.002000    4.495664e-07   
2            0.009001      0.000816         0.003000    1.296163e-06   
3            0.007333      0.000472         0.001999    1.798266e-06   
4            0.007668      0.000473         0.002335    4.712599e-04   
...               ...           ...              ...             ...   
307195       0.062797      0.002660         0.002502    4.096789e-04   
307196       0.061623      0.001415         0.002668    4.713736e-04   
307197       0.058948      0.003858         0.001669    4.703039e-04   
307198       0.055933      0.003531         0.002011    4.101811e-04   
307199       0.050085      0.005364         0.001842    6.281777e-04   

       param_colsample_bytree param_gamma param_learning_rate param_max_depth  \
0                         0.3           0             